# Basic Example

In [3]:
import torch
import confounding_robust_inference as cri

torch.random.manual_seed(0)

First, we create a synthetic data with binary response:

In [4]:
data = cri.data.SyntheticDataBinary()
Y, T, X, _, p_t, _ = data.sample(10)

AttributeError: module 'confounding_robust_inference' has no attribute 'data'

In [5]:
from confounding_robust_inference.data import SyntheticDataBinary